In [3]:
import torch
from torch import nn, optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error

/opt/conda/lib/python3.8/site-packages/pandas/compat/_optional.py:161: UserWarning: Pandas requires version '2.7.1' or newer of 'numexpr' (version '2.7.0' currently installed).
  warnings.warn(msg, UserWarning)


In [4]:
import json

with open('../data/preprocessed/3_month_retail.json', 'r') as f:
    data = json.load(f)

with open('../data/preprocessed/cluster_store.json','r') as f:
    cluster = json.load(f)
with open('../data/preprocessed/cluster_key_store_num.json', 'r') as f:
    cluster_key_store = json.load(f)

In [5]:
# cluster 번호를 지정해주면, 거기 안의 데이터를 X로 return 해주는 함수를 만들자
def cluster_dataset(cluster_num):
    data_tensor = torch.tensor([])
    for store_id in cluster[cluster_num]:
        sub_tensor = torch.tensor(list(data[str(store_id)].values()))
        data_tensor = torch.cat((data_tensor,sub_tensor),0)
    return data_tensor[:,:-1].t(), data_tensor[:,-1].t()

In [6]:
# define store_nums
store_list = [*range(1,643)]

In [13]:
class DSModel0(nn.Module):
    def __init__(self):
        super(DSModel0, self).__init__()
        
        self.ln = nn.LayerNorm(7848)
        self.conv_1 = nn.Conv2d(1, 1, kernel_size=(4,4))

        self.fc1 = nn.Linear(324, 64)
        self.fc2 = nn.Linear(64, 8)
        self.fc3 = nn.Linear(8,1)
    
    def forward(self, x):
        x = x.reshape(1, 1, 7848)
        x = self.ln(x)
        x = x.reshape(1, 1, 18, 436)
        x = F.relu(self.conv_1(x))
        x = F.max_pool2d(x, (4, 4))
        x = x.reshape(1,324)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        
        return x

class DSModel1(nn.Module):
    def __init__(self):
        super(DSModel1, self).__init__()
        
        self.ln = nn.LayerNorm(1098)
        self.conv_1 = nn.Conv2d(1, 1, kernel_size=(4,4))

        self.fc1 = nn.Linear(42, 8)
        self.fc2 = nn.Linear(8,1)
    
    def forward(self, x):
        x = x.reshape(1, 1, 1098)
        x = self.ln(x)
        x = x.reshape(1, 1, 18, 61)
        x = F.relu(self.conv_1(x))
        x = F.max_pool2d(x, (4, 4))
        x = x.reshape(1, 42)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        
        return x

class DSModel2(nn.Module):
    def __init__(self):
        super(DSModel2, self).__init__()
        
        self.ln = nn.LayerNorm(2610)
        self.conv_1 = nn.Conv2d(1, 1, kernel_size=(4,4))

        self.fc1 = nn.Linear(105, 32)
        self.fc2 = nn.Linear(32,8)
        self.fc3 = nn.Linear(8,1)
    
    def forward(self, x):
        x = x.reshape(1,1, 2610)
        x = self.ln(x)
        x = x.reshape(1, 1, 18, 145)
        x = F.relu(self.conv_1(x))
        x = F.max_pool2d(x, (4, 4))
        x = x.reshape(1,32)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        
        return x

In [14]:

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
result_tensor = torch.tensor([]).to(device)

for store_id in store_list:
    #상점이 어느 cluster에 해당하는지
    cluster_num = cluster_key_store[str(store_id)]
    # X = cluster_dataset(cluster_num)
    # X_train = X[:27]
    # X_test = X[27:]
    X_train, X_test = cluster_dataset(cluster_num)
    # X_train, X_test size : (28, ~)
    X_train = X_train.to(device)
    X_test = X_test.to(device)
    #product
    loss_ = {}
    for item in data[str(store_id)].keys():
        y = torch.tensor(data[str(store_id)][item][1:]).float()
        y_train = y 
        y_train = y_train.to(device)
        dataset_train = TensorDataset(X_train, y_train)
        train_dataloader = DataLoader(dataset_train, batch_size=1, shuffle=False)

        if cluster_num == '0':
            model = DSModel0()
        elif cluster_num == '1':
            model = DSModel1()
        elif cluster_num == '2':
            model = DSModel2()


        model = model.to(device)
        criterion = nn.MSELoss()
        optimizer = optim.Adam(model.parameters(), lr = 0.001, weight_decay=1e-7)

        # item_loss = 0.0
        for epoch in range(20):
            # running_loss = 0.0
            for i, data_input in enumerate(train_dataloader,0):
                inputs, value = data_input
                optimizer.zero_grad()
                outputs = model(inputs).flatten()
                loss = criterion(outputs, value)
                loss.backward()
                optimizer.step()

                # running_loss += loss.item()

        #item inference
        with torch.no_grad():
            y_pred = model(X_test)
        result_tensor = torch.cat((result_tensor, y_pred),0)
        



KeyboardInterrupt: 

In [ ]:
# result_tensor을 dataframe으로 전환
import pandas as pd 
import numpy as np

result_tensor_df = pd.DataFrame(result_tensor.detach().cpu().numpy())

#round시키고 int로 바꿔야함
result_tensor_df = result_tensor_df.round(0).astype(int)


In [6]:
result_tensor_df.to_csv('first_draft2.csv')

{'Bracket': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  1,
  0,
  0,
  0],
 'Brush': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  1,
  0,
  0],
 'Steak': [0,
  0,
  0,
  0,
  0,
  0,
  1,
  1,
  0,
  1,
  0,
  1,
  0,
  0,
  0,
  2,
  0,
  0,
  2,
  1,
  0,
  2,
  0,
  0,
  0,
  1,
  2,
  2,
  1],
 'Shrimp': [0,
  0,
  0,
  0,
  1,
  0,
  0,
  2,
  0,
  0,
  1,
  0,
  2,
  1,
  1,
  0,
  1,
  2,
  3,
  1,
  0,
  1,
  0,
  5,
  0,
  2,
  0,
  1,
  0],
 'Power Cord': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  1,
  0,
  0],
 'Phone Charger': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  2,
  3,
  7,
  1,
  1,
  2,
  8,
  8,
  8,
  4,
  7,
  4,
  6,
  0,
  1,
  1,
  0,
  2,
  2,
  1,
  0,
  0],
 'Paint': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
